# 1. Os 100 melhores filmes de todos os tempos

O IMDB mantém uma listagem dos 100 melhores filmes de todos tempos. Os filmes dessa lista são ranqueados de acordo com o seu sucesso (prêmios e nomeações), popularidade e sua grandiozidade do ponto de vista de direção e roteiro.

## 1.1 A base de dados

O arquivo [*top_100_greatest_movies.csv*](data/top_100_greatest_movies.csv) é uma planilha com duas colunas com o titúlo do filme e a sinopse do filme, nessa ordem.

## 1.2 O nosso objetivo

Agrupar os filmes de acordo com a semelhança entre suas sinopses.

# 2. Leitura e tratamento dos dados

As sinopses são textos em inglês de tamanho variável. Para extrair conhecimento dessa base de dados de forma automátizada temos que converter esse texto para um formato mais puro. Para tanto, o texto passará pelo seguinte processo de:
 - Remoção de stop words;
 - Stematização;
 - Remoção daquilo que não é palavra.
 
## 2.1 Leitura do Arquivo

In [1]:
import pandas as pd

# Leitura do csv
csv_data = pd.read_csv(
    "data/top_100_greatest_movies.csv",  # nome do arquivo
    header=None,  # informa que nao ha cabecalho no csv
    delimiter='\t'  # o delimitador de colunas
)

# Copiando os dados do csv
data = csv_data.values.copy()

# Fomato da base
print data.shape

# Exibindo as primeiras 5 linhas
csv_data[:5]

(100, 2)


,0,1
0,The Godfather,"In late summer 1945, guests are gathered for t..."
1,The Shawshank Redemption,"In 1947, Andy Dufresne (Tim Robbins), a banker..."
2,Schindler's List,The relocation of Polish Jews from surrounding...
3,Raging Bull,"The film opens in 1964, where an older and fat..."
4,Casablanca,"In the early years of World War II, December 1..."


### Separando titúlos de sinopses

In [2]:
# Coletando apenas os titulos
titles = data[:, 0]
print titles[17]

The Sound of Music


In [3]:
# Coeltando apenas as sinopses
synopses = data[:, 1]
print synopses[17]

The widowed, retired Austrian naval officer, Captain Von Trapp (Christopher Plummer) has made his Austrian home one of overly restrictive and harshly enforced discipline, one that, most unintentionally, causes his seven children to be underfed when it comes to joy and love. Being a nun living in a convent is similarly restrictive and unfulfilling for Maria (Julie Andrews), who breaks rules to try to change it. The reverend mother (Peggy Wood) decides that Maria, who is not cutting it as a nun, should leave and take on a job as governess at the nearby Von Trapp household in Salzburg.Through music and various outings, Maria gives the children a taste of a more fulfilling, joyous, life than they have ever known, and they come to love her very dearly. The Captain grows closer to his children, too, coming to understand the value and beauty of the freedoms that Maria has given them. Ironically, the freedom of all Austrians to live their lives to the fullest is in danger, for it is 1938, and 

### 2.1 Remoção de stop words

As stop words são palavras comuns no vocabulário que não agregam muita informação, como artigos e conjunções. Para a remoção será utilizado o módulo stopwords do nltk.corpus.

**OBSERVAÇÃO**: Se for a primeira execução do nltk descomente e excute a célula abaixo e faça o dowload do conjunto de stop words.

In [4]:
import nltk

# nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

#### Stopwords do Inglês:

In [5]:
stopwords = nltk.corpus.stopwords.words('english')
print stopwords[:10]

[u'i', u'me', u'my', u'myself', u'we', u'our', u'ours', u'ourselves', u'you', u'your']


#### Stopwords do Português:

In [6]:
stopwords_pt = nltk.corpus.stopwords.words('portuguese')
print stopwords_pt[:10]

[u'de', u'a', u'o', u'que', u'e', u'do', u'da', u'em', u'um', u'para']


### 2.2 Stemização

Em morfologia linguística e recuperação de informação a stemização (do inglês, stemming) é o processo de reduzir palavras flexionadas (ou às vezes derivadas) ao seu tronco (stem), base ou raiz, geralmente uma forma da palavra escrita.

Para stemização será utilizado a classe SnowballStemmer também do nltk

In [19]:
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer("english")  # tbm poderia ser 'portuguese'
stemmer.stem("Singing")

u'sing'

## 2.3 Lematização

In [8]:
from nltk.stem.wordnet import WordNetLemmatizer
lmtzr = WordNetLemmatizer()
lmtzr.lemmatize('cars')

u'car'

## 2.4 Uma função para tratar o texto

A partir de agora cada palavra cadeia de símbolos presentes no texto serão tratados como tokens. Essa função coleta as palavras válidas e as reduzem para seu formato bruto (stemização).

In [9]:
import re

def tokenize_and_stem(text):
    # Converte todos os elementos do texto em tokens.
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filtra tokens que não contem letras.
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    # converte tokens para seu formato bruto.
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

Testando a função

In [10]:
tokenize_and_stem('The Good muffins cost $3.88\nin New York.')

[u'the', u'good', u'muffin', u'cost', 'in', u'new', u'york']

# 3. Construção da matriz Tf-idf

O valor tf–idf (abreviação do inglês term frequency–inverse document frequency, que significa frequência do termo–inverso da frequência nos documentos), é uma medida estatística que tem o intuito de indicar a importância de uma palavra de um documento em relação a uma coleção de documentos ou em um corpus linguístico.

O valor td–idf de uma palavra aumenta proporcionalmente à medida que aumenta o número de ocorrências dela em um documento, no entanto, esse valor é equilibrado pela frequência da palavra no corpus. Isso auxilia a distinguir o fato da ocorrência de algumas palavras serem geralmente mais comuns que outras.

Para a construção da matriz será utilizada classe TfidfVectorizer

Que recebe como parâmetro:
 - stop_words: lista de stopwords;
 - tokenizer: função que converte o texto em tokens;
 - max_df: a máxima frequência que um termo tem que ter em todos os documentos para ser considerada válido e ser representado na matriz. Exemplo: se um termo aparece em mais de 80% (max_df=0.8) dos documentos, então ele é muito comum, trás pouco conhecimento e pode ser descartado;
 - min_df: ao contrário do max_df, define uma frequência mínima.

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

# criando o gerador da matriz:
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=200000,
                                   min_df=0.2, stop_words=stopwords,
                                   use_idf=True, tokenizer=tokenize_and_stem,
                                   ngram_range=(1,3))

# aplicando:
%time tfidf_matrix = tfidf_vectorizer.fit_transform(synopses) #fit the vectorizer to synopses

# imprimindo o formato:
tfidf_matrix.shape

CPU times: user 9.97 s, sys: 319 ms, total: 10.3 s
Wall time: 10.2 s


(100, 427)

In [12]:
# Listando os termos
terms = tfidf_vectorizer.get_feature_names()
print terms[:10]

[u"'d", u'abl', u'accept', u'accompani', u'across', u'act', u'action', u'admit', u'afterward', u'agre']


# 4. Agrupando com o K-means

De posse da matriz tf-idf podemos executar o algoritmo do K-Means. O K-Means começa analizando com um número pré definido de clusters. Cada observação é atribuida a um cluster e para minimizar the 
Each observation is assigned to a cluster (cluster assignment) so as to minimize the within cluster sum of squares. Next, the mean of the clustered observations is calculated and used as the new cluster centroid. Then, observations are reassigned to clusters and centroids recalculated in an iterative process until the algorithm reaches convergence.

In [13]:
from sklearn.cluster import KMeans

num_clusters = 5

km = KMeans(n_clusters=num_clusters)

%time km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

CPU times: user 183 ms, sys: 7.45 ms, total: 191 ms
Wall time: 194 ms


In [14]:
from sklearn.externals import joblib

# salva o modelo em um arquivo
joblib.dump(km,  'doc_cluster.pkl')

km = joblib.load('doc_cluster.pkl')
clusters = km.labels_.tolist()

#### Organizando a o resultado para melhor entendimento

In [15]:
import numpy as np

films = { 'title': titles, 'rank': range(1, len(titles) + 1), 'synopsis': synopses, 'cluster': clusters}
frame = pd.DataFrame(films, index = [clusters])
frame[:20]

,cluster,rank,synopsis,title
2,2,1,"In late summer 1945, guests are gathered for t...",The Godfather
3,3,2,"In 1947, Andy Dufresne (Tim Robbins), a banker...",The Shawshank Redemption
3,3,3,The relocation of Polish Jews from surrounding...,Schindler's List
0,0,4,"The film opens in 1964, where an older and fat...",Raging Bull
3,3,5,"In the early years of World War II, December 1...",Casablanca
3,3,6,"In 1963 Oregon, Randle Patrick McMurphy (Nicho...",One Flew Over the Cuckoo's Nest
3,3,7,"The film opens in Tara, a cotton plantation ow...",Gone with the Wind
4,4,8,"It's 1941, and newspaper tycoon Charles Foster...",Citizen Kane
1,1,9,Dorothy Gale (Judy Garland) is an orphaned tee...,The Wizard of Oz
4,4,10,"In 1996, treasure hunter Brock Lovett and his ...",Titanic


### Contagem por grupos

In [16]:
frame['cluster'].value_counts()

1    32
3    27
4    21
2    11
0     9
Name: cluster, dtype: int64

In [17]:
grouped = frame['rank'].groupby(frame['cluster'])

grouped.mean()

cluster
0    37.111111
1    57.718750
2    53.727273
3    35.148148
4    63.285714
Name: rank, dtype: float64

### Imprimindo a lista de grupos

In [18]:
print "Top terms per cluster:"
order_centroids = km.cluster_centers_.argsort()[:, ::-1]

for i in range(num_clusters):
    print "Cluster %d:" % (i + 1)

    top_terms = [terms[ind] for ind in order_centroids[i, :5]]
    print " Top Words: %s" % (top_terms)

    for title in frame.ix[i]['title'].values.tolist():
        print' - %s,' % title

Top terms per cluster:
Cluster 1:
 Top Words: [u'father', u'brother', u'son', u'fight', u'warn']
 - Raging Bull,
 - The Godfather: Part II,
 - 12 Angry Men,
 - Amadeus,
 - The Lord of the Rings: The Return of the King,
 - Gladiator,
 - Braveheart,
 - It Happened One Night,
 - Rain Man,
Cluster 2:
 Top Words: [u'tell', u'get', u"n't", u'car', u'say']
 - The Wizard of Oz,
 - Psycho,
 - Sunset Blvd.,
 - Vertigo,
 - On the Waterfront,
 - West Side Story,
 - E.T. the Extra-Terrestrial,
 - 2001: A Space Odyssey,
 - It's a Wonderful Life,
 - Some Like It Hot,
 - Unforgiven,
 - Rocky,
 - To Kill a Mockingbird,
 - My Fair Lady,
 - Ben-Hur,
 - The Exorcist,
 - The French Connection,
 - Tootsie,
 - Fargo,
 - The Green Mile,
 - Close Encounters of the Third Kind,
 - Nashville,
 - The Graduate,
 - American Graffiti,
 - Pulp Fiction,
 - The Maltese Falcon,
 - A Clockwork Orange,
 - Taxi Driver,
 - Rebel Without a Cause,
 - Rear Window,
 - The Third Man,
 - North by Northwest,
Cluster 3:
 Top Words: 

## Referências:
[1](http://stackoverflow.com/questions/27889873/clustering-text-documents-using-scikit-learn-kmeans-in-python) Clustering Text Documents Using Scikit-learn

[2](http://scikit-learn.org/stable/auto_examples/text/document_clustering.html) Document Clustering